<a href="https://colab.research.google.com/github/Diegoarmando24/Practica-2/blob/main/Ejercicio_22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

# ============================================
# MATRIZ BASE DEL EJERCICIO
# ============================================

# Matriz A dada (5x5) con estructura particular
A_original = np.array([
    [1, 0, 0, 0, 1],
    [-1, 1, 0, 0, 1],
    [-1, -1, 1, 0, 1],
    [-1, -1, -1, 1, 1],
    [-1, -1, -1, -1, 1]
], dtype=float)

# ============================================
# FUNCIÓN: Eliminación Gaussiana con pivoteo parcial
# ============================================

def Eliminacion_Gaussiana_Parcial(A, b):
    """
    Resuelve Ax = b usando eliminación Gaussiana con pivoteo parcial.
    Sustitución hacia atrás al final.
    """
    n = A.shape[0]
    A = A.copy()
    b = b.copy()

    for k in range(n - 1):
        # Selección de pivote máximo en columna k
        max_row = max(range(k, n), key=lambda i: abs(A[i][k]))
        if max_row != k:
            A[[k, max_row]] = A[[max_row, k]]
            b[[k, max_row]] = b[[max_row, k]]

        # Eliminación hacia abajo
        for i in range(k + 1, n):
            factor = A[i][k] / A[k][k]
            for j in range(k, n):
                A[i][j] -= factor * A[k][j]
            b[i] -= factor * b[k]

    # Sustitución hacia atrás
    x = np.zeros(n)
    for i in reversed(range(n)):
        suma = sum(A[i][j] * x[j] for j in range(i + 1, n))
        x[i] = (b[i] - suma) / A[i][i]
    return x

# ============================================
# FUNCIÓN: Norma infinito de un vector
# ============================================

def norma_inf(v):
    """
    Calcula la norma infinito (máximo valor absoluto) de un vector.
    """
    max_valor = 0.0
    for i in range(len(v)):
        if abs(v[i]) > max_valor:
            max_valor = abs(v[i])
    return max_valor

# ============================================
# FUNCIÓN: Número de condición en norma infinito
# ============================================

def condicion_inf(A):
    """
    Calcula el número de condición de la matriz A en norma infinito.
    Se hace una inversión aproximada usando eliminación Gaussiana para cada vector canónico.
    """
    n = A.shape[0]
    # Norma infinito de A
    norma_A = max(sum(abs(A[i][j]) for j in range(n)) for i in range(n))

    # Calcular inversa aproximada usando eliminación por columnas
    A_inv = np.zeros_like(A)
    for i in range(n):
        ei = np.zeros(n)
        ei[i] = 1
        A_inv[:, i] = Eliminacion_Gaussiana_Parcial(A, ei)

    # Norma infinito de A^{-1}
    norma_Ainv = max(sum(abs(A_inv[i][j]) for j in range(n)) for i in range(n))

    return norma_A * norma_Ainv

# ============================================
# FUNCIÓN: Permutaciones totales (filas y columnas)
# ============================================

def Permutaciones_Totales(A, b):
    """
    Aplica pivoteo total para resolver el sistema Ax = b.
    Devuelve matrices de permutación y matriz U transformada.
    """
    n = A.shape[0]
    U = A.copy()
    x = b.copy()
    P = np.eye(n)
    Q = np.eye(n)

    for k in range(n - 1):
        max_row, max_col = k, k
        max_val = abs(U[k][k])
        # Búsqueda del pivote más grande en submatriz restante
        for i in range(k, n):
            for j in range(k, n):
                if abs(U[i][j]) > max_val:
                    max_val = abs(U[i][j])
                    max_row, max_col = i, j

        # Intercambio de filas y columnas
        if max_row != k:
            U[[k, max_row]] = U[[max_row, k]]
            P[[k, max_row]] = P[[max_row, k]]
            x[[k, max_row]] = x[[max_row, k]]
        if max_col != k:
            U[:, [k, max_col]] = U[:, [max_col, k]]
            Q[:, [k, max_col]] = Q[:, [max_col, k]]

        # Eliminación hacia abajo
        for i in range(k + 1, n):
            factor = U[i][k] / U[k][k]
            for j in range(k, n):
                U[i][j] -= factor * U[k][j]

    return P, Q, U, x

# ============================================
# PRUEBA DEL SISTEMA CON DIFERENTES b
# ============================================

# Diferentes vectores b para probar estabilidad
b1 = np.array([1, 1, 1, 1, 1])
b2 = np.array([0, 0, 0, 0, 1])
b3 = np.array([1, 2, 3, 4, 5])
b4 = np.array([-1, 2, -3, 4, -5])
b5 = np.array([10, -10, 10, -10, 10])

# Ejecutar con cada vector b
x = Eliminacion_Gaussiana_Parcial(A_original, b1)
print("Solución para b1:", x)

x = Eliminacion_Gaussiana_Parcial(A_original, b2)
print("Solución para b2:", x)

x = Eliminacion_Gaussiana_Parcial(A_original, b3)
print("Solución para b3:", x)

x = Eliminacion_Gaussiana_Parcial(A_original, b4)
print("Solución para b4:", x)

x = Eliminacion_Gaussiana_Parcial(A_original, b5)
print("Solución para b5:", x)

# ============================================
# FACTORIZACIÓN LU CON PIVOTEO TOTAL
# ============================================

def LU_Pivoteo_Total(A):
    """
    Realiza la factorización LU con pivoteo total: A = P^T * L * U * Q^T
    """
    n = A.shape[0]
    U = A.copy()
    L = np.eye(n)
    P = np.eye(n)
    Q = np.eye(n)

    for k in range(n - 1):
        # Encontrar el pivote más grande
        max_row, max_col = k, k
        max_val = abs(U[k][k])
        for i in range(k, n):
            for j in range(k, n):
                if abs(U[i][j]) > max_val:
                    max_val = abs(U[i][j])
                    max_row, max_col = i, j

        # Intercambio de filas y columnas
        if max_row != k:
            U[[k, max_row]] = U[[max_row, k]]
            P[[k, max_row]] = P[[max_row, k]]
            L[[k, max_row], :k] = L[[max_row, k], :k]  # mantener coherencia en L

        if max_col != k:
            U[:, [k, max_col]] = U[:, [max_col, k]]
            Q[:, [k, max_col]] = Q[:, [max_col, k]]

        # Eliminación para construir L y U
        for i in range(k + 1, n):
            factor = U[i][k] / U[k][k]
            L[i][k] = factor
            for j in range(k, n):
                U[i][j] -= factor * U[k][j]

    return P, Q, L, U

# Verificación final: PAQ ≈ LU
P, Q, L, U = LU_Pivoteo_Total(A_original)
PAQ = P @ A_original @ Q
LU = L @ U
residuo = PAQ - LU

print("\nVerificación de PAQ = LU")
print("Norma infinita del residuo:", norma_inf(residuo.flatten()))
print("¿Es pequeña?", norma_inf(residuo.flatten()) < 1e-10)
